In [1]:
import xarray as xr 

ds = xr.open_dataset('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/01_raw/e16fe5f664d5290665a25e5afad51ce9.nc')

In [2]:
ds 

<xarray.Dataset> Size: 2GB
Dimensions:    (date: 96, latitude: 721, longitude: 1440)
Coordinates:
    number     int64 8B ...
  * date       (date) int64 768B 20150101 20150201 ... 20221101 20221201
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver     (date) <U4 2kB ...
Data variables:
    t2m        (date, latitude, longitude) float32 399MB ...
    sst        (date, latitude, longitude) float32 399MB ...
    tp         (date, latitude, longitude) float32 399MB ...
    pev        (date, latitude, longitude) float32 399MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-10T08:42 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
import numpy as np
from dinosaur import horizontal_interpolation
from dinosaur import spherical_harmonic
from dinosaur import xarray_utils
import neuralgcm

# Define source grid
source_lat = ds.latitude.values
source_lon = ds.longitude.values

source_grid = spherical_harmonic.Grid(
    latitude_nodes=len(source_lat),
    longitude_nodes=len(source_lon),
    latitude_spacing=xarray_utils.infer_latitude_spacing(ds.latitude),
    longitude_offset=xarray_utils.infer_longitude_offset(ds.longitude),
)

# Define target grid with 1° resolution
target_lat = np.arange(-90, 90, 1.0)  # From -90 to 90 degrees
target_lon = np.arange(0, 360, 1.0)     # From 0 to 359 degrees

target_grid = spherical_harmonic.Grid(
    latitude_nodes=len(target_lat),
    longitude_nodes=len(target_lon),
    latitude_spacing='gauss',              
    longitude_offset=0.0,               # Starting at 0 degrees
)

# Create the conservative regridder
regridder = horizontal_interpolation.ConservativeRegridder(
    source_grid=source_grid,
    target_grid=target_grid,
    skipna=True
)

# List of variables to regrid
variables_to_regrid = ['t2m', 'sst', 'tp', 'pev']

# Perform regridding for each variable
regridded_vars = {}
for var in variables_to_regrid:
    regridded_var = xarray_utils.regrid(ds[var], regridder)
    regridded_vars[var] = regridded_var

# Combine regridded variables into a new Dataset
ds_regridded = xr.Dataset(regridded_vars)



In [8]:
ds = ds_regridded

In [10]:
ds

<xarray.Dataset> Size: 100MB
Dimensions:    (date: 96, longitude: 360, latitude: 180)
Coordinates:
    number     int64 8B 0
  * date       (date) int64 768B 20150101 20150201 ... 20221101 20221201
    expver     (date) <U4 2kB '0001' '0001' '0001' ... '0001' '0001' '0001'
  * longitude  (longitude) float64 3kB 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
  * latitude   (latitude) float64 1kB -89.24 -88.25 -87.25 ... 87.25 88.25 89.24
Data variables:
    t2m        (date, longitude, latitude) float32 25MB 244.4 244.5 ... 250.7
    sst        (date, longitude, latitude) float32 25MB nan nan ... 271.5 271.5
    tp         (date, longitude, latitude) float32 25MB 6.809e-05 ... 0.0006224
    pev        (date, longitude, latitude) float32 25MB -6.448e-05 ... -1.49e-08

In [11]:
ds = ds.sel(latitude=slice(-30, 30))

In [12]:
ds

<xarray.Dataset> Size: 33MB
Dimensions:    (date: 96, longitude: 360, latitude: 60)
Coordinates:
    number     int64 8B 0
  * date       (date) int64 768B 20150101 20150201 ... 20221101 20221201
    expver     (date) <U4 2kB '0001' '0001' '0001' ... '0001' '0001' '0001'
  * longitude  (longitude) float64 3kB 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
  * latitude   (latitude) float64 480B -29.42 -28.42 -27.42 ... 28.42 29.42
Data variables:
    t2m        (date, longitude, latitude) float32 8MB 295.5 295.9 ... 288.8
    sst        (date, longitude, latitude) float32 8MB 296.7 297.1 ... nan nan
    tp         (date, longitude, latitude) float32 8MB 0.0005402 ... 7.781e-06
    pev        (date, longitude, latitude) float32 8MB 1.378e-07 ... 2.449e-06

In [13]:
import numpy as np
import xarray as xr

# Assuming your dataset is called ds
# Example dataset variables: t2m, sst, tp, pev

# 1. Extract relevant variables
variables = ['t2m', 'sst']
data_arrays = [ds[var] for var in variables]

# 2. Flatten latitude and longitude into a single node dimension
# Combine the latitude and longitude as one "node" dimension
data_arrays_flattened = [da.stack(node=('latitude', 'longitude')) for da in data_arrays]

# 3. Convert each variable's DataArray to a numpy array and add a new channel dimension
# (so we have the shape (time, nodes, channels))
target = np.stack([da.to_numpy() for da in data_arrays_flattened], axis=-1)


In [14]:
target

array([[[295.52103, 296.72638],
        [295.3564 , 296.57333],
        [295.18488, 296.36353],
        ...,
        [284.64005,       nan],
        [284.0082 ,       nan],
        [283.5111 ,       nan]],

       [[293.9972 , 295.8827 ],
        [293.95062, 295.79678],
        [293.89304, 295.71185],
        ...,
        [287.14087,       nan],
        [286.8251 ,       nan],
        [286.50452,       nan]],

       [[294.2437 , 295.67288],
        [294.16904, 295.69608],
        [294.08063, 295.67126],
        ...,
        [290.74597,       nan],
        [290.35205,       nan],
        [289.93826,       nan]],

       ...,

       [[289.91766, 290.86957],
        [289.9477 , 291.05774],
        [290.0198 , 291.27884],
        ...,
        [299.45245,       nan],
        [299.2676 ,       nan],
        [299.229  ,       nan]],

       [[290.98077, 292.3986 ],
        [291.01907, 292.55792],
        [290.9702 , 292.51703],
        ...,
        [292.16745,       nan],
        [291.55978

In [15]:
# Save as .npy file
np.save('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/target.npy', target)

In [16]:
# 1. Create a boolean mask where valid values are True and invalid (NaN) values are False
mask = ~np.isnan(target)

In [17]:
mask

array([[[ True,  True],
        [ True,  True],
        [ True,  True],
        ...,
        [ True, False],
        [ True, False],
        [ True, False]],

       [[ True,  True],
        [ True,  True],
        [ True,  True],
        ...,
        [ True, False],
        [ True, False],
        [ True, False]],

       [[ True,  True],
        [ True,  True],
        [ True,  True],
        ...,
        [ True, False],
        [ True, False],
        [ True, False]],

       ...,

       [[ True,  True],
        [ True,  True],
        [ True,  True],
        ...,
        [ True, False],
        [ True, False],
        [ True, False]],

       [[ True,  True],
        [ True,  True],
        [ True,  True],
        ...,
        [ True, False],
        [ True, False],
        [ True, False]],

       [[ True,  True],
        [ True,  True],
        [ True,  True],
        ...,
        [ True, False],
        [ True, False],
        [ True, False]]])

In [18]:
# Save as .npy file
np.save('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/mask.npy', mask)

In [19]:
import numpy as np
import xarray as xr

# Assuming your dataset is called ds
# Example dataset variables: t2m, sst, tp, pev

# 1. Extract relevant variables
variables = ['tp', 'pev']
data_arrays = [ds[var] for var in variables]

# 2. Flatten latitude and longitude into a single node dimension
# Combine the latitude and longitude as one "node" dimension
data_arrays_flattened = [da.stack(node=('latitude', 'longitude')) for da in data_arrays]

# 3. Convert each variable's DataArray to a numpy array and add a new channel dimension
# (so we have the shape (time, nodes, channels))
covariates = np.stack([da.to_numpy() for da in data_arrays_flattened], axis=-1)


In [20]:
covariates

array([[[ 5.4019596e-04,  1.3783574e-07],
        [ 4.6423243e-04,  1.3783573e-07],
        [ 3.7716696e-04,  1.3783574e-07],
        ...,
        [ 3.4839970e-05, -1.2828368e-03],
        [ 4.7743248e-05, -4.5946045e-04],
        [ 8.6343563e-05,  2.0427792e-06]],

       [[ 6.6785852e-04, -5.9604645e-08],
        [ 6.0978951e-04, -5.9604645e-08],
        [ 5.6171190e-04, -5.9604645e-08],
        ...,
        [ 2.7000555e-04, -1.7224486e-03],
        [ 1.8245151e-04, -6.3110661e-04],
        [ 1.3260679e-04, -5.9450713e-07]],

       [[ 6.0515048e-04, -7.4505806e-08],
        [ 5.4533209e-04, -7.4505799e-08],
        [ 5.1349151e-04, -7.4505806e-08],
        ...,
        [ 9.7678731e-06, -2.3830666e-03],
        [ 4.3324681e-06, -8.6646917e-04],
        [ 7.0577444e-06, -1.1298209e-06]],

       ...,

       [[ 4.0827808e-04,  1.4901161e-08],
        [ 4.4418321e-04,  1.4901161e-08],
        [ 4.7704202e-04,  1.4901161e-08],
        ...,
        [ 5.8611422e-05, -2.9069418e-03],
     

In [21]:
# Save as .npy file
np.save('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/covariates.npy', covariates)

In [22]:
import pandas as pd
import os

def generate_and_save_metadata(df: pd.DataFrame, lat_col: str = 'lat', lon_col: str = 'lon', save_directory: str = None) -> pd.DataFrame:
    """
    Generates metadata from a given DataFrame by identifying unique latitude and longitude
    combinations and assigning a unique node ID to each combination. The metadata is then
    saved to a specified Parquet file.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - lat_col (str): The name of the column containing latitude data.
    - lon_col (str): The name of the column containing longitude data.
    - save_directory (str, optional): The directory where the metadata file will be saved. If None, 
                                      the file will be saved in the current working directory.

    Returns:
    - pd.DataFrame: A DataFrame containing the metadata with latitude, longitude, and 'node_id' as columns.
    """
    # Ensure the DataFrame contains necessary columns
    if lat_col not in df.columns or lon_col not in df.columns:
        raise ValueError(f"DataFrame must contain '{lat_col}' and '{lon_col}' columns.")

    # Create a unique node ID for each unique latitude-longitude combination
    unique_lat_lon = df[[lat_col, lon_col]].drop_duplicates().reset_index(drop=True)
    unique_lat_lon['node_id'] = unique_lat_lon.index

    # Create the metadata DataFrame
    metadata = unique_lat_lon.set_index('node_id')

    # Handle save directory and save metadata to Parquet file
    if save_directory:
        os.makedirs(save_directory, exist_ok=True)  # Create directory if it doesn't exist
        file_path = os.path.join(save_directory, "metadata.parquet")
    else:
        file_path = "metadata.parquet"  # Save in the current working directory

    metadata.to_parquet(file_path)
    print(f"Metadata file saved at: {file_path}")

    return metadata


In [23]:
import pandas as pd
import os

# Assume you've already stacked your data arrays as 'data_arrays_flattened'
# We'll use one of the flattened DataArrays to extract the node information
da_flattened = data_arrays_flattened[0]  # Using the first variable for example

# Get the MultiIndex from the 'node' dimension
node_index = da_flattened.indexes['node']

# Extract latitude and longitude from the MultiIndex
latitudes = node_index.get_level_values('latitude').values
longitudes = node_index.get_level_values('longitude').values

# Create a DataFrame with 'lat' and 'lon' columns
df = pd.DataFrame({
    'lat': latitudes,
    'lon': longitudes
})

# Now use your 'generate_and_save_metadata' function
metadata = generate_and_save_metadata(
    df, 
    lat_col='lat', 
    lon_col='lon', 
    save_directory='/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/'
)


Metadata file saved at: /Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/metadata.parquet


In [24]:
# metadata

In [25]:
import pandas as pd
metadata = pd.read_parquet("/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/metadata.parquet")

In [26]:
metadata

,lat,lon
node_id,,
0,-29.418159,0.0
1,-29.418159,1.0
2,-29.418159,2.0
3,-29.418159,3.0
4,-29.418159,4.0
...,...,...
21595,29.418159,355.0
21596,29.418159,356.0
21597,29.418159,357.0


In [27]:
# Converting the DataFrame to a numpy ndarray
metadata_array = metadata.to_numpy()


In [28]:
metadata_array

array([[-29.41815859,   0.        ],
       [-29.41815859,   1.        ],
       [-29.41815859,   2.        ],
       ...,
       [ 29.41815859, 357.        ],
       [ 29.41815859, 358.        ],
       [ 29.41815859, 359.        ]])

In [29]:
# Save as .npy file
np.save('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/metadata.npy', metadata_array)

In [45]:
from tsl.ops.similarities import geographical_distance
# Calculate geographical distances with coordinates converted to radians.
dist = geographical_distance(metadata_array, to_rad=True)

In [46]:
dist

array([[   0.        ,   96.85708835,  193.71239696, ..., 6550.04816429,
        6545.74977196, 6543.1693716 ],
       [  96.85708835,    0.        ,   96.85708835, ..., 6556.06114557,
        6550.04816429, 6545.74977196],
       [ 193.71239696,   96.85708835,    0.        , ..., 6563.78397026,
        6556.06114557, 6550.04816429],
       ...,
       [6550.04816429, 6556.06114557, 6563.78397026, ...,    0.        ,
          96.85708835,  193.71239696],
       [6545.74977196, 6550.04816429, 6556.06114557, ...,   96.85708835,
           0.        ,   96.85708835],
       [6543.1693716 , 6545.74977196, 6550.04816429, ...,  193.71239696,
          96.85708835,    0.        ]])

In [47]:
# Save as .npy file
np.save('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/distances.npy', dist)

In [48]:
from typing import Optional, Union, List

from tsl.datasets.prototypes import TabularDataset

class DroughtDataset(TabularDataset):

    similarity_options = {'distance', 'correlation'}

    def __init__(self,
                 root: str = None
                 ):

        self.root = root

        # Load data
        target, mask, u, dist, metadata = self.load()

        covariates = {
            'u': (u),
            'metadata' : (metadata),
            'distances': (dist)
        }

        super().__init__(target=target,
                         mask=mask,
                         covariates=covariates,
                         similarity_score='distance',
                         temporal_aggregation='mean',
                         spatial_aggregation='mean',
                         name='DroughtDataset')

    def load(self):
        """
        Load data from files.

        Returns:
            tuple: Containing dataframe, mask, metadata, and distance matrix.
        """
        target_path = f"{self.root}target.npy"
        mask_path = f"{self.root}mask.npy"
        dist_path = f"{self.root}distances.npy"
        covariates_path = f"{self.root}covariates.npy"
        metadata = f"{self.root}metadata.npy"

        # Load main data
        target = np.load(target_path)
        mask = np.load(mask_path)
        u = np.load(covariates_path)
        dist = np.load(dist_path)
        metadata = np.load(dist_path)

        return target, mask, u, dist, metadata

    def compute_similarity(self, method: str, **kwargs):
        """
        Compute similarity matrix based on the specified method.

        Args:
            method (str): The similarity computation method ('distance' or 'correlation').
            **kwargs: Additional keyword arguments for similarity computation.

        Returns:
            numpy.ndarray: Computed similarity matrix.

        Raises:
            ValueError: If an unknown similarity method is provided.
        """
        if method == "distance":
            # Calculate a Gaussian kernel similarity from the distance matrix, using a default or provided 'theta'
            theta = kwargs.get('theta', np.std(self.distances))
            return self.gaussian_kernel(self.distances, theta=theta)
        elif method == "correlation":
            # Compute the average correlation between nodes over the target features
            # Reshape target data to have nodes as columns
            target_values = self.target.values.reshape(len(self.target), -1, len(self.target_node_feature))
            # Average over the target features
            target_mean = target_values.mean(axis=2)
            # Compute correlation between nodes
            corr = np.corrcoef(target_mean, rowvar=False)
            return (corr + 1) / 2  # Normalize to [0, 1]
        else:
            raise ValueError(f"Unknown similarity method: {method}")

    @staticmethod
    def gaussian_kernel(distances, theta):
        """
        Compute Gaussian kernel similarity from distances.

        Args:
            distances (numpy.ndarray): Distance matrix.
            theta (float): Kernel bandwidth parameter.

        Returns:
            numpy.ndarray: Gaussian kernel similarity matrix.
        """
        return np.exp(-(distances ** 2) / (2 * (theta ** 2)))

In [49]:
dataset = DroughtDataset(root='/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/')

In [50]:
dataset.target

array([[[295.52103, 296.72638],
        [295.3564 , 296.57333],
        [295.18488, 296.36353],
        ...,
        [284.64005,       nan],
        [284.0082 ,       nan],
        [283.5111 ,       nan]],

       [[293.9972 , 295.8827 ],
        [293.95062, 295.79678],
        [293.89304, 295.71185],
        ...,
        [287.14087,       nan],
        [286.8251 ,       nan],
        [286.50452,       nan]],

       [[294.2437 , 295.67288],
        [294.16904, 295.69608],
        [294.08063, 295.67126],
        ...,
        [290.74597,       nan],
        [290.35205,       nan],
        [289.93826,       nan]],

       ...,

       [[289.91766, 290.86957],
        [289.9477 , 291.05774],
        [290.0198 , 291.27884],
        ...,
        [299.45245,       nan],
        [299.2676 ,       nan],
        [299.229  ,       nan]],

       [[290.98077, 292.3986 ],
        [291.01907, 292.55792],
        [290.9702 , 292.51703],
        ...,
        [292.16745,       nan],
        [291.55978

In [51]:
print(f"Has missing values: {dataset.has_mask}")

Has missing values: True


In [52]:
dataset.mask

array([[[ True,  True],
        [ True,  True],
        [ True,  True],
        ...,
        [ True, False],
        [ True, False],
        [ True, False]],

       [[ True,  True],
        [ True,  True],
        [ True,  True],
        ...,
        [ True, False],
        [ True, False],
        [ True, False]],

       [[ True,  True],
        [ True,  True],
        [ True,  True],
        ...,
        [ True, False],
        [ True, False],
        [ True, False]],

       ...,

       [[ True,  True],
        [ True,  True],
        [ True,  True],
        ...,
        [ True, False],
        [ True, False],
        [ True, False]],

       [[ True,  True],
        [ True,  True],
        [ True,  True],
        ...,
        [ True, False],
        [ True, False],
        [ True, False]],

       [[ True,  True],
        [ True,  True],
        [ True,  True],
        ...,
        [ True, False],
        [ True, False],
        [ True, False]]])

In [53]:
dataset.covariates

{'u': array([[[ 5.4019596e-04,  1.3783574e-07],
         [ 4.6423243e-04,  1.3783573e-07],
         [ 3.7716696e-04,  1.3783574e-07],
         ...,
         [ 3.4839970e-05, -1.2828368e-03],
         [ 4.7743248e-05, -4.5946045e-04],
         [ 8.6343563e-05,  2.0427792e-06]],
 
        [[ 6.6785852e-04, -5.9604645e-08],
         [ 6.0978951e-04, -5.9604645e-08],
         [ 5.6171190e-04, -5.9604645e-08],
         ...,
         [ 2.7000555e-04, -1.7224486e-03],
         [ 1.8245151e-04, -6.3110661e-04],
         [ 1.3260679e-04, -5.9450713e-07]],
 
        [[ 6.0515048e-04, -7.4505806e-08],
         [ 5.4533209e-04, -7.4505799e-08],
         [ 5.1349151e-04, -7.4505806e-08],
         ...,
         [ 9.7678731e-06, -2.3830666e-03],
         [ 4.3324681e-06, -8.6646917e-04],
         [ 7.0577444e-06, -1.1298209e-06]],
 
        ...,
 
        [[ 4.0827808e-04,  1.4901161e-08],
         [ 4.4418321e-04,  1.4901161e-08],
         [ 4.7704202e-04,  1.4901161e-08],
         ...,
         [ 5

In [54]:
dataset.distances

array([[   0.      ,   96.857086,  193.7124  , ..., 6550.0483  ,
        6545.75    , 6543.1694  ],
       [  96.857086,    0.      ,   96.857086, ..., 6556.061   ,
        6550.0483  , 6545.75    ],
       [ 193.7124  ,   96.857086,    0.      , ..., 6563.784   ,
        6556.061   , 6550.0483  ],
       ...,
       [6550.0483  , 6556.061   , 6563.784   , ...,    0.      ,
          96.857086,  193.7124  ],
       [6545.75    , 6550.0483  , 6556.061   , ...,   96.857086,
           0.      ,   96.857086],
       [6543.1694  , 6545.75    , 6550.0483  , ...,  193.7124  ,
          96.857086,    0.      ]], dtype=float32)

In [55]:
sim = dataset.compute_similarity("distance")  # or dataset.compute_similarity()

In [56]:
sim

array([[1.        , 0.999815  , 0.9992604 , ..., 0.42914304, 0.42961958,
        0.42990583],
       [0.999815  , 1.        , 0.999815  , ..., 0.42847675, 0.42914304,
        0.42961958],
       [0.9992604 , 0.999815  , 1.        , ..., 0.42762148, 0.42847675,
        0.42914304],
       ...,
       [0.42914304, 0.42847675, 0.42762148, ..., 1.        , 0.999815  ,
        0.9992604 ],
       [0.42961958, 0.42914304, 0.42847675, ..., 0.999815  , 1.        ,
        0.999815  ],
       [0.42990583, 0.42961958, 0.42914304, ..., 0.9992604 , 0.999815  ,
        1.        ]], dtype=float32)

In [57]:
# Adjust connectivity to reduce the number of edges
connectivity = dataset.get_connectivity(
    knn=5,                
    include_self=False,
    normalize_axis=1,
    layout="edge_index"
)

In [58]:
edge_index, edge_weight = connectivity

In [61]:
from tsl.data import SpatioTemporalDataset

# covariates=dict(u=dataset.covariates['u'])
covariates=dataset.covariates
mask = dataset.get_mask().copy()

torch_dataset = SpatioTemporalDataset(target=dataset.dataframe(),
                                      mask=mask,
                                      covariates=covariates,
                                      connectivity=connectivity,
                                      horizon=6, # Predict 7 step ahead
                                      window=18, # Use 30 timestamps to predict the next one
                                      stride=1 # Move 7 step forward each time
                                      )

In [62]:
from tsl.data.preprocessing import StandardScaler, MinMaxScaler

scalers = {
    'target': MinMaxScaler(axis=(0, 1)),
    'u': MinMaxScaler(axis=(0, 1))
}


In [63]:
from tsl.data.datamodule import (SpatioTemporalDataModule,
                                 TemporalSplitter)
                                 
# Split data sequentially:
#   |------------ dataset -----------|
#   |--- train ---|- val -|-- test --|
splitter = TemporalSplitter(val_len=0.15, test_len=0.15)

In [65]:
# Create a SpatioTemporalDataModule
datamodule = SpatioTemporalDataModule(
    dataset=torch_dataset,
    scalers=scalers,
    mask_scaling=True,
    splitter=splitter,
    batch_size=2,
    workers=16,
    )

print(datamodule)

SpatioTemporalDataModule(train_len=None, val_len=None, test_len=None, scalers=[target, u], batch_size=2)


In [66]:
from typing import Optional

import torch
from torch import nn, Tensor
from torch_geometric.nn import MessagePassing
from tsl.nn.blocks import RNNBase
from tsl.nn.layers import Dense, GraphGRUCellBase, Activation


class GraphAnisoConv(MessagePassing):
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: int = 1,
                 edge_dim: Optional[int] = None,
                 activation: str = 'leaky_relu'):
        super(GraphAnisoConv, self).__init__(aggr="add", node_dim=-2)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        self.msg_mlps = nn.ModuleList([
            nn.Sequential(
                nn.Linear(2 * (in_channels if i == 0 else out_channels),
                          out_channels),
                Activation(activation),
                nn.Linear(out_channels, out_channels),
            )
            for i in range(kernel_size)
        ])

        edge_dim = edge_dim or 1  # accommodate for edge_weight
        self.lin_edge = nn.Linear(edge_dim, out_channels, bias=False)

        self.gate_mlp = Dense(out_channels, 1, activation='sigmoid')

        self.skip_conn = nn.Linear(in_channels, out_channels)
        self.activation = Activation(activation)

    def forward(self, x, edge_index, edge_attr: Optional[Tensor] = None):
        """"""
        out, x_ = 0, x
        for idx in range(self.kernel_size):
            x_ = self.propagate(edge_index, idx=idx, x=x_, edge_attr=edge_attr)
            out += x_
        out = self.activation(out + self.skip_conn(x))
        return out

    def message(self, x_i, x_j, idx, edge_attr: Optional[Tensor] = None):
        mij = self.msg_mlps[idx](torch.cat([x_i, x_j], -1))
        if edge_attr is not None:
            if edge_attr.ndim == 1:  # accommodate for edge_weight
                edge_attr = edge_attr.view(-1, 1)
            mij = mij + self.lin_edge(edge_attr)
        return self.gate_mlp(mij) * mij


class GraphAnisoGRUCell(GraphGRUCellBase):

    def __init__(self, input_size: int, hidden_size: int,
                 edge_dim: Optional[int] = None,
                 activation: str = 'leaky_relu'):
        self.input_size = input_size
        # instantiate gates
        forget_gate = GraphAnisoConv(input_size + hidden_size, hidden_size,
                                     edge_dim=edge_dim, activation=activation)
        update_gate = GraphAnisoConv(input_size + hidden_size, hidden_size,
                                     edge_dim=edge_dim, activation=activation)
        candidate_gate = GraphAnisoConv(input_size + hidden_size, hidden_size,
                                        edge_dim=edge_dim,
                                        activation=activation)
        super(GraphAnisoGRUCell, self).__init__(hidden_size=hidden_size,
                                                forget_gate=forget_gate,
                                                update_gate=update_gate,
                                                candidate_gate=candidate_gate)


class GraphAnisoGRU(RNNBase):

    def __init__(self, input_size: int, hidden_size: int,
                 edge_dim: Optional[int] = None,
                 n_layers: int = 1, cat_states_layers: bool = False,
                 return_only_last_state: bool = False,
                 activation: str = 'leaky_relu'):
        self.input_size = input_size
        self.hidden_size = hidden_size
        rnn_cells = [
            GraphAnisoGRUCell(input_size if i == 0 else hidden_size,
                              hidden_size, edge_dim=edge_dim,
                              activation=activation)
            for i in range(n_layers)
        ]
        super(GraphAnisoGRU, self).__init__(rnn_cells, cat_states_layers,
                                            return_only_last_state)


In [ ]:
from typing import Optional, Union, List

import torch
from torch import Tensor, nn
from torch_geometric.typing import Adj
from tsl.nn.blocks import MLPDecoder
from tsl.nn.layers import MultiLinear, NodeEmbedding
from tsl.nn.models import BaseModel
from tsl.nn.utils import maybe_cat_exog
from tsl.utils import ensure_list


def maybe_cat_emb(x: Tensor, emb: Optional[Tensor]):
    if emb is None:
        return x
    if emb.ndim < x.ndim:
        emb = emb[[None] * (x.ndim - emb.ndim)]
    emb = emb.expand(*x.shape[:-1], -1)
    return torch.cat([x, emb], dim=-1)


class STGNN(BaseModel):
    available_embedding_pos = {'encoding', 'decoding'}

    def __init__(self, input_size: int, horizon: int,
                 n_nodes: int = None,
                 output_size: int = None,
                 exog_size: int = 0,
                 hidden_size: int = 32,
                 emb_size: int = 0,
                 add_embedding_before: Optional[
                     Union[str, List[str]]] = 'encoding',
                 use_local_weights: Union[str, List[str]] = None,
                 activation: str = 'elu'):
        super(STGNN, self).__init__()

        self.input_size = input_size
        self.horizon = horizon
        self.n_nodes = n_nodes
        self.output_size = output_size or input_size
        self.hidden_size = hidden_size
        self.exog_size = exog_size
        self.activation = activation

        # EMBEDDING
        if add_embedding_before is None:
            add_embedding_before = set()
        else:
            add_embedding_before = set(ensure_list(add_embedding_before))
            if not add_embedding_before.issubset(self.available_embedding_pos):
                raise ValueError("Parameter 'add_embedding_before' must be a "
                                 f"subset of {self.available_embedding_pos}")
        self.add_embedding_before = add_embedding_before

        if emb_size > 0:
            self.emb = NodeEmbedding(n_nodes, emb_size)
        else:
            self.register_module('emb', None)

        # ENCODER
        self.encoder_input = input_size + exog_size
        if 'encoding' in self.add_embedding_before and self.emb is not None:
            self.encoder_input += emb_size

        if use_local_weights is not None:
            self.use_local_weights = set(ensure_list(use_local_weights))
            if len(self.use_local_weights.difference(['encoder', 'decoder'])):
                raise ValueError("Parameter 'use_local_weights' must be "
                                 "'encoder', 'decoder', or both.")
        else:
            self.use_local_weights = set()

        if 'encoder' in self.use_local_weights:
            self.encoder = MultiLinear(self.encoder_input, hidden_size, n_nodes)
        else:
            self.encoder = nn.Linear(self.encoder_input, hidden_size)

        # DECODER
        self.decoder_input = hidden_size
        if 'decoding' in self.add_embedding_before and self.emb is not None:
            self.decoder_input += emb_size
        if 'decoder' in self.use_local_weights:
            raise NotImplementedError()
        else:
            self.decoder = MLPDecoder(input_size=self.decoder_input,
                                      hidden_size=self.hidden_size,
                                      output_size=self.output_size,
                                      horizon=self.horizon,
                                      activation=self.activation)

    def stmp(self, x: Tensor, edge_index: Adj,
             edge_weight: Optional[Tensor] = None,
             emb: Optional[Tensor] = None) -> Tensor:
        raise NotImplementedError

    def forward(self, x: Tensor, edge_index: Adj,
                edge_weight: Optional[Tensor] = None,
                u: Optional[Tensor] = None,
                node_idx: Optional[Tensor] = None) -> Tensor:
        """"""
        # x: [batches steps nodes features]
        x = maybe_cat_exog(x, u)
        batch_size = x.size(0)
        emb = self.emb(expand=(batch_size, -1, -1),
                       node_index=node_idx) if self.emb is not None else None

        if 'encoding' in self.add_embedding_before and emb is not None:
            x = maybe_cat_emb(x, emb[:, None])

        # ENCODER   ###########################################################
        x = self.encoder(x)

        # SPATIOTEMPORAL MESSAGE-PASSING   ####################################
        out = self.stmp(x, edge_index, edge_weight, emb)

        # DECODER   ###########################################################
        if 'decoding' in self.add_embedding_before:
            out = maybe_cat_emb(out, emb)

        out = self.decoder(out)

        return out


class TimeThenSpace(STGNN):
    available_embedding_pos = {'encoding', 'message_passing', 'decoding'}

    def __init__(self, input_size: int, horizon: int,
                 temporal_encoder: nn.Module,
                 spatial_encoder: Union[nn.Module, List[nn.Module]],
                 n_nodes: int = None,
                 output_size: int = None,
                 exog_size: int = 0,
                 hidden_size: int = 32,
                 emb_size: int = 0,
                 add_embedding_before: Union[str, List[str]] = 'encoding',
                 use_local_weights: Union[str, List[str]] = None,
                 activation: str = 'elu'):
        super(TimeThenSpace, self).__init__(input_size=input_size,
                                            horizon=horizon,
                                            n_nodes=n_nodes,
                                            output_size=output_size,
                                            exog_size=exog_size,
                                            hidden_size=hidden_size,
                                            emb_size=emb_size,
                                            add_embedding_before=add_embedding_before,
                                            use_local_weights=use_local_weights,
                                            activation=activation)
        # STMP
        self.temporal_encoder = temporal_encoder
        if not isinstance(spatial_encoder, nn.ModuleList):
            spatial_encoder = nn.ModuleList(ensure_list(spatial_encoder))
        self.mp_layers = spatial_encoder
        self.spatial_layers = len(self.mp_layers)

    def stmp(self, x: Tensor, edge_index: Adj,
             edge_weight: Optional[Tensor] = None,
             emb: Optional[Tensor] = None) -> Tensor:
        # temporal encoding
        out = self.temporal_encoder(x)
        # spatial encoding
        for layer in self.mp_layers:
            if 'message_passing' in self.add_embedding_before:
                out = maybe_cat_emb(out, emb)
            out = layer(out, edge_index, edge_weight)
        return out


class TimeAndSpace(STGNN):

    def __init__(self, input_size: int, horizon: int, stmp_conv: nn.Module,
                 n_nodes: int = None,
                 output_size: int = None,
                 exog_size: int = 0,
                 hidden_size: int = 32,
                 emb_size: int = 0,
                 add_embedding_before: Union[str, List[str]] = 'encoding',
                 use_local_weights: Union[str, List[str]] = None,
                 activation: str = 'elu'):
        super(TimeAndSpace, self).__init__(input_size=input_size,
                                           horizon=horizon,
                                           n_nodes=n_nodes,
                                           output_size=output_size,
                                           exog_size=exog_size,
                                           hidden_size=hidden_size,
                                           emb_size=emb_size,
                                           add_embedding_before=add_embedding_before,
                                           use_local_weights=use_local_weights,
                                           activation=activation)

        # STMP
        self.stmp_conv = stmp_conv

    def stmp(self, x: Tensor, edge_index: Adj,
             edge_weight: Optional[Tensor] = None,
             emb: Optional[Tensor] = None) -> Tensor:
        # spatiotemporal encoding
        out = self.stmp_conv(x, edge_index, edge_weight)
        return out


In [ ]:
from typing import Union, List

class TimeAndGraphAnisoModel(TimeAndSpace):

    def __init__(self, input_size: int, horizon: int, n_nodes: int = None,
                 output_size: int = None,
                 exog_size: int = 0,
                 hidden_size: int = 32,
                 emb_size: int = 0,
                 add_embedding_before: Union[str, List[str]] = 'encoding',
                 use_local_weights: Union[str, List[str]] = None,
                 n_layers: int = 1,
                 activation: str = 'elu'):
        stmp_conv = GraphAnisoGRU(input_size=hidden_size,
                                  hidden_size=hidden_size,
                                  n_layers=n_layers,
                                  activation=activation,
                                  return_only_last_state=True)
        super(TimeAndGraphAnisoModel, self).__init__(
            input_size=input_size,
            horizon=horizon,
            stmp_conv=stmp_conv,
            n_nodes=n_nodes,
            output_size=output_size,
            exog_size=exog_size,
            hidden_size=hidden_size,
            emb_size=emb_size,
            add_embedding_before=add_embedding_before,
            use_local_weights=use_local_weights,
            activation=activation
        )


In [ ]:
hidden_size = 64          # Number of hidden units
ff_size = 128             # Number of units in the feed-forward layers
n_layers = 3              # Number of SpatioTemporalConvNet blocks
temporal_kernel_size = 3  # Size of the temporal convolution kernel
spatial_kernel_size = 3   # Order of the spatial diffusion process
norm='layer'
gated=True

input_size = torch_dataset.n_channels
n_nodes = torch_dataset.n_nodes
horizon = torch_dataset.horizon
exog_size = torch_dataset.input_map.u.shape[-1] if 'u' in torch_dataset else 0

In [ ]:
model = TimeAndGraphAnisoModel(
    input_size=input_size,
    horizon=horizon,
    n_nodes=n_nodes,
    output_size=input_size,
    exog_size=exog_size,
    hidden_size=hidden_size
)

# Print the model architecture
print(model)

In [ ]:
def print_model_size(model):
    tot = sum([p.numel() for p in model.parameters() if p.requires_grad])
    out = f"Number of model ({model.__class__.__name__}) parameters:{tot:10d}"
    print("=" * len(out))
    print(out)

In [ ]:
print_model_size(model)

In [ ]:
from tsl.metrics.torch import MaskedMSE, MaskedMAE, MaskedMAPE
from tsl.engines import Predictor

# Define the loss function
loss_fn = MaskedMSE()

# Setup metrics
metrics = {
    'mse': MaskedMSE(),
    'mae': MaskedMAE(),
    'mape': MaskedMAPE(),
    'mse_at_3': MaskedMSE(at=2),  # '2' indicates the third time step
    'mse_at_6': MaskedMSE(at=5)
}

# Setup predictor
predictor = Predictor(
    model=model,
    optim_class=torch.optim.Adam,
    optim_kwargs={'lr': 0.001},
    loss_fn=loss_fn,
    metrics=metrics
)


In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

early_stop_callback = EarlyStopping(
    monitor='val_mse',
    patience=5,
    mode='min'
)

checkpoint_callback = ModelCheckpoint(
    dirpath='logs',
    save_top_k=1,
    monitor='val_mse',  # Change this to 'val_mse'
    mode='min',
)

# Setup trainer
trainer = pl.Trainer(max_epochs=100,
                    #  logger=logger,
                    #  limit_train_batches=100,  # end an epoch after 200 updates
                     callbacks=[early_stop_callback, checkpoint_callback],
                     log_every_n_steps=2,
                     gradient_clip_val=1.0,    # Prevent exploding gradients
                     precision=16
                     )


In [ ]:
# Set float32 matmul precision to 'medium' or 'high'
torch.set_float32_matmul_precision('medium')

In [ ]:
trainer.fit(predictor, datamodule=datamodule)

In [ ]:
predictor.load_model(checkpoint_callback.best_model_path)
predictor.freeze()

In [ ]:
trainer.test(predictor, datamodule=datamodule)

In [ ]:
trainer.validate(predictor, datamodule=datamodule)

In [ ]:
predictions_test = trainer.predict(predictor, datamodule.test_dataloader())

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import pandas as pd
from matplotlib.colors import Normalize

def visualize_predictions(predictions, metadata, idx, horizon=None, 
                         batches_to_visualize=None, variables_to_visualize=None):
    """
    Visualize predicted vs actual values for specified batches and variables with variable-specific normalization.

    Parameters:
    - predictions (dict): Contains 'y' and 'y_hat' tensors.
    - metadata (pd.DataFrame): DataFrame with 'lat' and 'lon' columns.
    - idx (int): Index of the item being visualized.
    - horizon (int, optional): Number of time steps to visualize. Defaults to the maximum horizon in data.
    - batches_to_visualize (list or range, optional): List of batch indices to visualize. Defaults to all batches.
    - variables_to_visualize (list or range, optional): List of variable indices to visualize. Defaults to all variables.
    """
    # Extract predictions and actual values
    y_hat = predictions['y_hat'].squeeze().numpy()  # Shape: [batch, horizon, spatial_dim, variables] or [batch, horizon, spatial_dim]
    y = predictions['y'].squeeze().numpy()          # Shape: [batch, horizon, spatial_dim, variables] or [batch, horizon, spatial_dim]

    # Determine the horizon if not provided
    if horizon is None:
        horizon = y_hat.shape[1]

    # Determine the number of variables
    if y_hat.ndim == 3:  # Handle 3D tensor case
        num_variables = 1
        variables_to_visualize = [0]  # Only one variable to visualize
    else:
        num_variables = y_hat.shape[-1]  # Handle 4D tensor case
        if variables_to_visualize is None:
            variables_to_visualize = list(range(num_variables))

    # Validate variables_to_visualize
    variables_to_visualize = [var for var in variables_to_visualize if var < num_variables]
    if not variables_to_visualize:
        raise ValueError("No valid variables to visualize. Check 'variables_to_visualize' indices.")

    # Get latitude and longitude from metadata
    lats = metadata['lat'].values
    lons = metadata['lon'].values

    # Precompute min and max for each variable in variables_to_visualize
    var_min_max = {}
    for var in variables_to_visualize:
        if num_variables == 1:
            var_data_y = y
            var_data_y_hat = y_hat
        else:
            var_data_y = y[..., var]
            var_data_y_hat = y_hat[..., var]
        min_val = min(np.min(var_data_y), np.min(var_data_y_hat))
        max_val = max(np.max(var_data_y), np.max(var_data_y_hat))
        # Handle case where min and max are equal to avoid division by zero
        if min_val == max_val:
            min_val -= 1  # Arbitrary adjustment
            max_val += 1
        var_min_max[var] = (min_val, max_val)

    # Color scheme
    cmap = plt.cm.RdYlBu_r  # You can choose other colormaps as well

    # If batches_to_visualize is None, visualize all batches
    if batches_to_visualize is None:
        batches_to_visualize = range(y_hat.shape[0])

    for batch in batches_to_visualize:
        for var in variables_to_visualize:
            fig, axes = plt.subplots(horizon, 2, figsize=(20, 5 * horizon),
                                     subplot_kw={'projection': ccrs.PlateCarree()})
            fig.subplots_adjust(hspace=0.4, wspace=0.2)

            # Retrieve normalization for the current variable
            min_val, max_val = var_min_max[var]
            norm = Normalize(vmin=min_val, vmax=max_val)

            for week in range(horizon):
                ax_hat = axes[week, 0]
                ax_y = axes[week, 1]

                # Extract the specific variable's data
                if num_variables == 1:
                    z_hat = y_hat[batch, week, :]  # 3D case
                    z_y = y[batch, week, :]        # 3D case
                else:
                    z_hat = y_hat[batch, week, :, var]  # 4D case
                    z_y = y[batch, week, :, var]        # 4D case

                # Plot y_hat (Predicted) with larger dots
                scatter_hat = ax_hat.scatter(
                    lons, lats, c=z_hat, cmap=cmap, norm=norm,
                    transform=ccrs.PlateCarree(), s=50,  # Increased from 10 to 50
                    edgecolor='k', linewidth=0.1
                )
                ax_hat.add_feature(cfeature.COASTLINE)
                ax_hat.add_feature(cfeature.BORDERS, linestyle=':')
                ax_hat.set_extent([lons.min(), lons.max(), lats.min(), lats.max()], crs=ccrs.PlateCarree())
                cbar_hat = plt.colorbar(scatter_hat, ax=ax_hat, orientation='vertical', pad=0.02, aspect=16, shrink=0.8)
                cbar_hat.set_label('Predicted EDDI')
                ax_hat.set_title(f'Batch {batch + 1}, Variable {var + 1}, Week {week + 1} - Predicted')

                # Plot y (Actual) with larger dots
                scatter_y = ax_y.scatter(
                    lons, lats, c=z_y, cmap=cmap, norm=norm,
                    transform=ccrs.PlateCarree(), s=50,  # Increased from 10 to 50
                    edgecolor='k', linewidth=0.1
                )
                ax_y.add_feature(cfeature.COASTLINE)
                ax_y.add_feature(cfeature.BORDERS, linestyle=':')
                ax_y.set_extent([lons.min(), lons.max(), lats.min(), lats.max()], crs=ccrs.PlateCarree())
                cbar_y = plt.colorbar(scatter_y, ax=ax_y, orientation='vertical', pad=0.02, aspect=16, shrink=0.8)
                cbar_y.set_label('Actual EDDI')
                ax_y.set_title(f'Batch {batch + 1}, Variable {var + 1}, Week {week + 1} - Actual')

            fig.suptitle(f'Item {idx + 1}, Batch {batch + 1}, Variable {var + 1} - Predicted vs Actual EDDI', fontsize=16)
            plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust rect to prevent overlap with suptitle

            # Display the figure
            plt.show()
            plt.close(fig)

# Example Usage

# Assuming you have the metadata DataFrame available
metadata = pd.read_parquet('ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/metadata.parquet')

# Specify the indices of the items you want to visualize
items_to_visualize = [1]  # Replace with desired item indices (0-based indexing)
batches_to_visualize = [0, 1]  # Replace with desired batch indices within each item
variables_to_visualize = [0]  # Replace with desired variable indices (e.g., if multiple EDDI metrics)

# Call the function for specified items in predictions_test
for idx in items_to_visualize:
    pred = predictions_test[idx]
    visualize_predictions(
        pred, metadata, idx, horizon=pred['y'].shape[1],
        batches_to_visualize=batches_to_visualize, 
        variables_to_visualize=variables_to_visualize
    )
    print(f"Processed item {idx + 1}/{len(predictions_test)}")


In [ ]:
import plotly.graph_objects as go
import plotly.subplots as sp
import pandas as pd
import numpy as np

def visualize_predictions_plotly(predictions, metadata, idx, horizon=None, 
                                 batches_to_visualize=None, variables_to_visualize=None):
    """
    Visualize predicted vs actual values using Plotly for specified batches and variables with variable-specific normalization.
    
    Parameters:
    - predictions (dict): Contains 'y' and 'y_hat' tensors.
    - metadata (pd.DataFrame): DataFrame with 'lat' and 'lon' columns.
    - idx (int): Index of the item being visualized.
    - horizon (int, optional): Number of time steps to visualize. Defaults to the maximum horizon in data.
    - batches_to_visualize (list or range, optional): List of batch indices to visualize. Defaults to all batches.
    - variables_to_visualize (list or range, optional): List of variable indices to visualize. Defaults to all variables.
    """
    # Extract predictions and actual values
    y_hat = predictions['y_hat'].squeeze().numpy()  # Shape: [batch, horizon, spatial_dim, variables]
    y = predictions['y'].squeeze().numpy()          # Shape: [batch, horizon, spatial_dim, variables]

    # Ensure y_hat and y have the same shape
    assert y_hat.shape == y.shape, "Predictions and actual values must have the same shape"

    # Determine the horizon if not provided
    if horizon is None:
        horizon = y_hat.shape[1]

    # Determine the number of variables
    if y_hat.ndim == 3:
        # Shape: [batch, horizon, spatial_dim]
        num_variables = 1
        y_hat = y_hat[..., np.newaxis]  # Add a variables dimension
        y = y[..., np.newaxis]
    else:
        num_variables = y_hat.shape[-1]

    if variables_to_visualize is None:
        variables_to_visualize = list(range(num_variables))
    else:
        # Ensure the variable indices are within the correct range
        variables_to_visualize = [v for v in variables_to_visualize if v < num_variables]
        if not variables_to_visualize:
            raise ValueError("No valid variables to visualize. Check 'variables_to_visualize' indices.")

    # Get latitude and longitude from metadata
    lats = metadata['lat'].values
    lons = metadata['lon'].values

    # Ensure that the number of spatial points matches the number of lats and lons
    spatial_dim = y_hat.shape[2]
    if spatial_dim != len(lats):
        raise ValueError(f"The number of spatial points in predictions ({spatial_dim}) does not match the number of locations in metadata ({len(lats)}).")

    # Calculate the min and max for lats and lons for setting map extent
    lat_min, lat_max = lats.min(), lats.max()
    lon_min, lon_max = lons.min(), lons.max()

    # If batches_to_visualize is None, visualize all batches
    if batches_to_visualize is None:
        batches_to_visualize = range(y_hat.shape[0])
    else:
        # Ensure the batch indices are within the correct range
        batches_to_visualize = [b for b in batches_to_visualize if b < y_hat.shape[0]]
        if not batches_to_visualize:
            raise ValueError("No valid batches to visualize. Check 'batches_to_visualize' indices.")

    # Precompute min and max for each variable in variables_to_visualize
    var_min_max = {}
    for var in variables_to_visualize:
        var_data_y = y[..., var]  # Shape: [batch, horizon, spatial_dim]
        var_data_y_hat = y_hat[..., var]  # Shape: [batch, horizon, spatial_dim]
        min_val = min(np.min(var_data_y), np.min(var_data_y_hat))
        max_val = max(np.max(var_data_y), np.max(var_data_y_hat))
        # Handle case where min and max are equal to avoid division by zero
        if min_val == max_val:
            min_val -= 1  # Arbitrary adjustment
            max_val += 1
        var_min_max[var] = (min_val, max_val)

    # If variables_to_visualize is [2], ensure that variable index 2 exists
    # (0-based indexing)
    for var in variables_to_visualize:
        if var >= num_variables:
            raise ValueError(f"Variable index {var} is out of range for data with {num_variables} variables.")

    # Loop through each batch and variable
    for batch in batches_to_visualize:
        for var in variables_to_visualize:
            frames = []
            slider_steps = []

            # Retrieve normalization for the current variable
            min_val, max_val = var_min_max[var]

            # Create frames for animation
            for week in range(horizon):
                # Predicted data for current week
                pred_data = go.Scattergeo(
                    lon=lons,
                    lat=lats,
                    mode='markers',
                    marker=dict(
                        size=6,
                        color=y_hat[batch, week, :, var],
                        colorscale='RdYlBu_r',
                        cmin=min_val,
                        cmax=max_val,
                        colorbar=dict(title='Predicted EDDI'),
                        opacity=0.8
                    ),
                    name='Predicted',
                    showlegend=False
                )

                # Actual data for current week
                actual_data = go.Scattergeo(
                    lon=lons,
                    lat=lats,
                    mode='markers',
                    marker=dict(
                        size=6,
                        color=y[batch, week, :, var],
                        colorscale='RdYlBu_r',
                        cmin=min_val,
                        cmax=max_val,
                        colorbar=dict(title='Actual EDDI'),
                        opacity=0.8
                    ),
                    name='Actual',
                    showlegend=False
                )

                # Define frame data
                frame = go.Frame(
                    data=[pred_data, actual_data],
                    name=f'Week {week + 1}'
                )
                frames.append(frame)

                # Create slider step for this frame
                slider_step = dict(
                    method='animate',
                    args=[[f'Week {week + 1}'],
                          dict(mode='immediate',
                               frame=dict(duration=500, redraw=True),
                               transition=dict(duration=0))],
                    label=f'Week {week + 1}'
                )
                slider_steps.append(slider_step)

            # Initial data (first frame)
            pred_data_initial = go.Scattergeo(
                lon=lons,
                lat=lats,
                mode='markers',
                marker=dict(
                    size=6,
                    color=y_hat[batch, 0, :, var],
                    colorscale='RdYlBu_r',
                    cmin=min_val,
                    cmax=max_val,
                    colorbar=dict(title='Predicted EDDI'),
                    opacity=0.8
                ),
                name='Predicted',
                showlegend=True
            )

            actual_data_initial = go.Scattergeo(
                lon=lons,
                lat=lats,
                mode='markers',
                marker=dict(
                    size=6,
                    color=y[batch, 0, :, var],
                    colorscale='RdYlBu_r',
                    cmin=min_val,
                    cmax=max_val,
                    colorbar=dict(title='Actual EDDI'),
                    opacity=0.8
                ),
                name='Actual',
                showlegend=True
            )

            # Create subplots with two geo subplots side by side
            fig = sp.make_subplots(
                rows=1, cols=2,
                specs=[[{'type': 'scattergeo'}, {'type': 'scattergeo'}]],
                subplot_titles=("Predicted EDDI", "Actual EDDI")
            )

            # Add initial traces to respective subplots
            fig.add_trace(pred_data_initial, row=1, col=1)
            fig.add_trace(actual_data_initial, row=1, col=2)

            # Add frames
            fig.frames = frames

            # Define sliders
            fig.update_layout(
                title_text=f'Item {idx + 1}, Batch {batch + 1}, Variable {var + 1} - Predicted vs Actual EDDI',
                updatemenus=[dict(
                    type='buttons',
                    showactive=False,
                    buttons=[dict(label='Play',
                                  method='animate',
                                  args=[None, {"frame": {"duration": 500, "redraw": True},
                                               "fromcurrent": True,
                                               "transition": {"duration": 0}}],
                                  )],
                    x=0.1,
                    y=1.15
                )],
                sliders=[dict(
                    active=0,
                    pad={"t": 50},
                    steps=slider_steps,
                    currentvalue=dict(
                        visible=True,
                        prefix="Week: ",
                        xanchor="right",
                        font=dict(size=14, color="#666")
                    ),
                )],
                geo=dict(
                    scope='world',
                    projection_type='natural earth',
                    showland=True,
                    landcolor='lightgray',
                    showcountries=True,
                    countrycolor='black',
                    lataxis=dict(range=[lat_min - 1, lat_max + 1]),
                    lonaxis=dict(range=[lon_min - 1, lon_max + 1]),
                ),
                geo2=dict(
                    scope='world',
                    projection_type='natural earth',
                    showland=True,
                    landcolor='lightgray',
                    showcountries=True,
                    countrycolor='black',
                    lataxis=dict(range=[lat_min - 1, lat_max + 1]),
                    lonaxis=dict(range=[lon_min - 1, lon_max + 1]),
                )
            )

            # Update frames to assign traces to correct subplots
            for frame in fig.frames:
                week_idx = int(frame.name.split()[1]) - 1
                frame.data = [
                    go.Scattergeo(
                        lon=lons,
                        lat=lats,
                        mode='markers',
                        marker=dict(
                            size=6,
                            color=y_hat[batch, week_idx, :, var],
                            colorscale='RdYlBu_r',
                            cmin=min_val,
                            cmax=max_val,
                            opacity=0.8
                        ),
                        showlegend=False
                    ),
                    go.Scattergeo(
                        lon=lons,
                        lat=lats,
                        mode='markers',
                        marker=dict(
                            size=6,
                            color=y[batch, week_idx, :, var],
                            colorscale='RdYlBu_r',
                            cmin=min_val,
                            cmax=max_val,
                            opacity=0.8
                        ),
                        showlegend=False
                    )
                ]

            # Update layout for better appearance
            fig.update_layout(
                height=600,
                width=1200,
                margin=dict(l=50, r=50, t=100, b=50)
            )

            # Display the figure
            fig.show()

            # Optionally, save the figure to an HTML file
            # fig.write_html(f'item_{idx+1}_batch_{batch+1}_variable_{var+1}.html')

# Example Usage

# Assuming you have the metadata DataFrame available
metadata = pd.read_parquet('ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/metadata.parquet')

# Ensure that predictions_test is a list of dictionaries with 'y' and 'y_hat'
# For example:
# predictions_test = [{'y': tensor1, 'y_hat': tensor2}, {'y': tensor3, 'y_hat': tensor4}, ...]

# Specify the indices of the items you want to visualize
items_to_visualize = [1]  # Replace with desired item indices (0-based indexing)
batches_to_visualize = [0, 1]  # Replace with desired batch indices within each item
variables_to_visualize = [0]  # Replace with desired variable indices (e.g., if multiple EDDI metrics)

# Call the function for specified items in predictions_test
for idx in items_to_visualize:
    pred = predictions_test[idx]
    visualize_predictions_plotly(
        pred, metadata, idx, horizon=pred['y'].shape[1],
        batches_to_visualize=batches_to_visualize, 
        variables_to_visualize=variables_to_visualize
    )
    print(f"Processed item {idx + 1}/{len(predictions_test)}")
